In [86]:
import Pkg

#Pkg.add("Zygote")

using Zygote
using Flux
using Flux: onehotbatch, onecold, crossentropy
using MLDatasets
#using CUDA
using Statistics

LoadError: IOError: open("/Users/tommystell/.julia/logs/manifest_usage.toml.pid", 2562, 292): no space left on device (ENOSPC)

In [68]:
train_x, train_y = MNIST(split=:train)[:]
test_x, test_y = MNIST(split=:test)[:]

train_x = float.(train_x) ./ 255.0
test_x = float.(test_x) ./ 255.0

train_x = reshape(train_x, 28, 28, 1, :);
test_x = reshape(test_x, 28, 28, 1, :);

train_y = onehotbatch(train_y, 0:9);
test_y = onehotbatch(test_y, 0:9);


In [75]:
conv1 = Conv((3,3), 1=>16, relu)
pool1 = MaxPool((2,2))
conv2 = Conv((3,3), 16=>32, relu)
pool2 = MaxPool((2,2))
conv3 = Conv((3,3), 32=>64, relu)
pool3 = MaxPool((2,2))
flatten = Flux.flatten
dense1 = Dense(64, 128, relu)
dense2 = Dense(128, 10)
softmax_layer = softmax

model = Chain(conv1, pool1, conv2, pool2, conv3, pool3, flatten, dense1, dense2, softmax_layer)

loss(x,y) = crossentropy(model(x), y)
opt = ADAM()

Adam(eta=0.001, beta=(0.9, 0.999), epsilon=1.0e-8)

In [76]:
function visualize_layers(model, x)
    println("Input shape: ", size(x))
    x = conv1(x); println("After Conv1: ", size(x))
    x = pool1(x); println("After Pool1: ", size(x))
    x = conv2(x); println("After Conv2: ", size(x))
    x = pool2(x); println("After Pool2: ", size(x))
    x = flatten(x); println("After Flatten: ", size(x))
    x = dense1(x); println("After Dense1: ", size(x))
    x = dense2(x); println("After Dense2: ", size(x))
    x = softmax_layer(x); println("After Softmax: ", size(x))
end


visualize_layers (generic function with 1 method)

In [84]:
function train_model!(model, train_X, train_Y, opt, epochs, batch_size)
    data_loader = Flux.DataLoader((train_X, train_Y), batchsize=batch_size, shuffle=true)
    
    opt_state = Flux.setup(opt, model)  

    for epoch in 1:epochs
        for (x, y) in data_loader
            gs = gradient(() -> Flux.Losses.mse(model(x), y), Flux.params(model))
            Flux.update!(opt_state, model, gs)  
        end
        println("Epoch $epoch complete")
    end
end


train_model! (generic function with 5 methods)

In [85]:
train_model!(model, train_x, train_y, opt, 5, 64)

visualize_layers(model, train_x[:, :, :, 1:1])

accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))
println("Test Accuracy: ", accuracy(test_x, test_y))


┌ Warning: Implicit gradients such as `gradient(f, ::Params)` are deprecated in Flux!
│ Please see the docs for new explicit form.
│   caller = train_model!(model::Chain{Tuple{Conv{2, 4, typeof(relu), Array{Float32, 4}, Vector{Float32}}, MaxPool{2, 4}, Conv{2, 4, typeof(relu), Array{Float32, 4}, Vector{Float32}}, MaxPool{2, 4}, Conv{2, 4, typeof(relu), Array{Float32, 4}, Vector{Float32}}, MaxPool{2, 4}, typeof(Flux.flatten), Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}, typeof(softmax)}}, train_X::Array{Float64, 4}, train_Y::OneHotArrays.OneHotMatrix{UInt32, Vector{UInt32}}, opt::Adam{Float64, Tuple{Float64, Float64}, Float64}, epochs::Int64, batch_size::Int64) at In[84]:8
└ @ Main ./In[84]:8


LoadError: type Grads has no field layers